In [1]:
import agents_env
from agents_env.agent_mimic_train import HumanoidEnvTrain
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *
from copy import deepcopy

In [2]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [3]:
#this is just dummy data to initialize the agent
trajectory = SimpleConverter('motions/humanoid3d_walk.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid.xml'

data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = jp.asarray(trajectory.data_pos)
data_vel_mocap = jp.asarray(trajectory.data_vel)

data_dict_mocap = trajectory.duration_dict

data_xpos_mocap=jp.asarray(trajectory.data_xpos)
data_xrot_mocap=jp.asarray(trajectory.data_xrot)

In [4]:
#get th kp and kd for the agent
kp,kd = generate_kp_kd_gains()
print(kp)
print(kd)

[1000 1000 1000  100  100  100  400  400  400  300  400  400  400  300
  500  500  500  500  400  400  400  500  500  500  500  400  400  400]
[100 100 100  10  10  10  40  40  40  30  40  40  40  30  50  50  50  50
  40  40  40  50  50  50  50  40  40  40]


In [5]:
envs.register_environment('humanoidEnvMimicEval',HumanoidEnvTrain)
env_name = 'humanoidEnvMimicEval'
env = envs.get_environment(env_name=env_name,
                           reference_trajectory_qpos=data_pos_mocap,
                           reference_trajectory_qvel = data_vel_mocap,
                            duration_trajectory=trajectory.total_time,
                            dict_duration= data_dict_mocap,
                           model_path=model_path,
                           kp_gains = kp,
                           kd_gains = kd,
                           reference_x_pos=data_xpos_mocap,
                           reference_x_rot=data_xrot_mocap)
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step_custom)


#select the pd_control
from agents_env.pds_controllers_agents import stable_pd_controller_custom_trajectory, stable_pd_controller_trajectory,stable_pd_controller_action,feedback_pd_controller
#env.set_pd_callback(stable_pd_controller_custom_trajectory)
env.set_pd_callback(stable_pd_controller_trajectory)
#env.set_pd_callback(feedback_pd_controller)

In [6]:
from some_math.math_utils import generate_trajectory,compute_cubic_trajectory,start_trajectories

In [7]:
#since we are going to use custom trajectory, we will set up the initial
#and the end time
t_init = 1
t_end = 3

In [8]:
trajec_dict = dict()


a_jnt_right =get_actuator_indx(env.sys.mj_model,'right_shoulder','Y')
a_jnt_left =get_actuator_indx(env.sys.mj_model,'left_shoulder','X')
a_jnt_left_elbow = get_actuator_indx(env.sys.mj_model,'left_elbow','X')
a_jnt_right_elbow = get_actuator_indx(env.sys.mj_model,'right_elbow','X')

right_knee = get_actuator_indx(env.sys.mj_model,'right_knee','X')
left_knee = get_actuator_indx(env.sys.mj_model,'left_knee','X')

trajec_dict[a_jnt_right] = generate_trajectory(t_init,t_end, 0, -1.5)
#trajectory left
trajec_dict[a_jnt_left] = generate_trajectory(t_init, t_end, 0, 1.5)
# #left elbow   
trajec_dict[a_jnt_left_elbow]= generate_trajectory(t_init, t_end, 0, 1.5)
# #right elbow
trajec_dict[a_jnt_right_elbow] = generate_trajectory(t_init, t_end, 0, 1.5)

trajec_dict[right_knee] = generate_trajectory(t_init, t_end, 0, 0)

trajec_dict[left_knee] = generate_trajectory(t_init, t_end, 0, 0)


start_trajec = start_trajectories(trajec_dict)

In [9]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500

In [10]:
for i in range(n_steps):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    #print('time: ',state.pipeline_state.time)
    time = jp.clip(time, t_init, t_end)
    
    # print(target)    
    # print(target.shape)    
    state = jit_step(state, ctrl,start_trajec,time)
    
    rollout.append(state.pipeline_state)

rewards: -0.0004901019856333733
rewards: -0.0007646052399650216
rewards: -0.0015031943330541253
rewards: -0.0020952639169991016
rewards: -0.002120229182764888
rewards: -0.00259069399908185
rewards: -0.0032346027437597513
rewards: -0.002789794933050871
rewards: -0.003557255258783698
rewards: -0.0038039612118154764
rewards: -0.027431098744273186
rewards: -0.025614337995648384
rewards: -0.02598680928349495
rewards: -0.024930547922849655
rewards: -0.024496393278241158
rewards: -0.02443564124405384
rewards: -0.02345086820423603
rewards: -0.022775093093514442
rewards: -0.022900031879544258
rewards: -0.022330567240715027
rewards: -0.022692734375596046
rewards: -0.021906422451138496
rewards: -0.021747536957263947
rewards: -0.02122875489294529
rewards: -0.021181745454669
rewards: -0.02070053480565548
rewards: -0.019461840391159058
rewards: -0.01897614635527134
rewards: -0.019212152808904648
rewards: -0.02378811314702034
rewards: -0.014730630442500114
rewards: -0.014757970348000526
rewards: -0.0

In [12]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

This browser does not support the video tag.
